In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("test-twist-3D.geo")

Info    : Reading 'test-twist-3D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.000410522s, CPU 0.000399s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.022223s, CPU 0.022213s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.00438163s, CPU 0.003394s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0096335s, CPU 0.008645s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.

In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 302)


LowLevelFEM.Problem("test-twist-3D", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 2648)

In [4]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
ff(x, y, z) = √(y^2 + z^2)
load = FEM.load("load", fy=ff)

("load", 0, Main.ff, 0)

In [5]:
n1x(x, y, z) = y
n1y(x, y, z) = z
n2x(x, y, z) = -z
n2y = n1x
cs = FEM.CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = FEM.rotateNodes(problem, "body", cs)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  7939, 7940, 7939, 7940, 7941, 7942, 7943, 7942, 7943, 7944], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  7939, 7939, 7940, 7940, 7941, 7942, 7942, 7943, 7943, 7944], [2.4492935982947065e-17, 1.0, -1.0, 2.4492935982947065e-17, 1.0, 0.9107991560957914, 0.41284972720736285, -0.4128497272073629, 0.9107991560957914, 0.9999999999999998  …  0.1687220986435244, 0.9856636614126164, -0.9856636614126164, 0.16872209864352436, 1.0, 0.015058119843009733, 0.9998866200858942, -0.9998866200858942, 0.015058119843009735, 0.9999999999999999], 7944, 7944), 2648, 3)

In [6]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 4.5950432758863755; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f3D, LowLevelFEM.Problem("test-twist-3D", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 2648))

In [7]:
K = Q' * K * Q

7944×7944 SparseArrays.SparseMatrixCSC{Float64, Int64} with 567122 stored entries:
⎡⣿⣿⡀⣰⣿⣿⣳⣶⣦⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⣟⣧⣶⣿⣿⣒⣶⣶⣶⣗⣶⣶⣶⣶⣶⣷⣇⣿⣿⣿⣿⎤
⎢⢀⣨⣿⣿⣿⣇⣠⣠⣠⣦⣽⡿⠿⣷⣶⣶⣤⣤⣀⠀⣀⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣧⎥
⎢⣿⣿⠿⢿⣿⣿⣷⣿⣿⣿⡿⠟⠻⠿⠿⠿⣿⣿⣿⣷⣦⣿⣿⣿⣿⡿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢹⣾⠀⣺⣽⣿⠻⣦⣳⢿⣃⠀⠀⠀⠀⠀⠈⣐⡮⠉⠀⣿⣇⣶⢖⣵⣤⣆⡞⣤⢶⡧⢞⢾⣿⡿⢻⣿⡻⡿⎥
⎢⠈⢻⠠⣾⣿⣿⣽⣞⣿⣿⣇⠀⠀⠀⢠⠤⣶⢾⣾⠄⠛⣿⡿⣷⣦⣶⣿⣾⡥⣗⢿⣾⣾⢟⣻⢿⢭⣟⣻⣯⎥
⎢⠀⠘⣷⡿⣿⠏⠉⠘⠉⠙⠻⣦⡀⠀⣤⠆⣜⡆⠩⠀⠀⣿⡿⠋⠿⣷⡿⡥⣭⡃⡉⣷⢸⣿⣻⣏⣝⣿⢫⡟⎥
⎢⠀⠀⢿⣧⣿⡆⠀⠀⠀⠀⠀⠈⣿⣿⣇⡀⡠⡫⠄⠀⠀⣿⣷⢰⣾⠂⡰⣿⡻⣟⡗⣾⣼⣿⣿⣻⣽⣪⡷⠅⎥
⎢⠀⠀⢸⣿⣿⡇⠀⠀⠀⡖⠠⠟⠉⠹⣻⣾⣿⣟⣿⠀⠀⣿⣿⣹⡝⡻⣿⢻⣿⣿⣻⠿⣷⣮⣿⣹⣿⣿⢿⡧⎥
⎢⠀⠀⠀⣿⣿⣿⢂⢠⣸⣟⠲⠽⡤⡪⣿⢿⣿⣿⣿⠄⠀⣿⡿⣯⣯⣿⡿⣭⣸⣽⢿⣿⣳⡜⣿⣿⣿⣿⣿⣾⎥
⎢⣤⢤⠀⠘⢿⣿⡎⠋⠚⠟⠃⠂⠀⠁⠛⠛⠛⠟⠻⣦⡄⣿⣯⠷⠺⠓⠚⢻⠿⠞⣼⡗⠛⡬⡟⣽⣿⡿⣷⣷⎥
⎢⢩⣿⣤⣼⣬⣿⣤⣤⣿⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣭⣿⣿⣧⣼⣤⣼⣤⣤⣧⣤⣿⣿⣿⣿⣼⣯⣯⣼⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⢩⣽⢿⣯⡿⠋⢙⣛⣟⣻⡿⣯⢯⡟⣉⣿⣿⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⢿⣟⣯⣻⎥
⎢⢸⣼⣿⣿⣿⡿⢜⣵⢨⣿⢿⣧⠺⠛⣷⡩⣯⣿⢾⠂⣀⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⣿⣷⡟⣿⣻⢽⎥
⎢⢸⣿⣿⣿⣿⣷⠠⢿⣻⣿⠟⡯⣴⣮⣿⣛⡟⣯⣾⣀⠀⣿⣿⣿⣿⣿⣿⣿⣏⣿⣿⣿⣿⣿⣿⣿⣿⣿⣻⣭⎥
⎢⢹⣽⣿⣿⣿⣿⠚⣭⢥⢯⠧⠻⣿⢮⣿⣿⣖⣾⣻⠇⠉⣿⣿⣿⣿⣿⣯⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⢸⣿⣿⣿⣿⣿⠼⡷⣻⣷⢧⣬⣹⣭⣿⡞⣿⣷⢶⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣞⎥
⎢⢸⣿⣾⣿⣿⣿⣺⣕⣾⢟⣶⣶⣶⣿⡹⣿⣙⠾⡛⡤⣿⣿⣿⣿⣿⡿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⠽⢿⣿⣿⣿⣿⣿⡿⣿⣞⡿⢾⣿⣻⣟⣻⣿⣿⣟⣭⡶⣿⣟⣿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣶⣧⢷⣷⣽⡳⣻⣿⣿⣿⣿⣿⡿⣋⣿⣿⢷⣿⣭⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⠿⣿⣿⣿⣿⡮⡿⣾⣯⠶⠝⠏⠿⡷⣻⣿⢽⣿⣿⣿⣯⣻⣟⣞⡟⣾⣿⣿⣻⢿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [8]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [9]:
q = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [1.3578071810835735e12; -2.598270000069679e12; … ; -2.2606706741193403e12; 1.0128123028409521e12;;], [0.0], Int64[], 1, :u3D, LowLevelFEM.Problem("test-twist-3D", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 2648))

In [10]:
q1 = Q * q

LowLevelFEM.VectorField(Matrix{Float64}[], [2.598270000069679e12; 1.3578071810835735e12; … ; 1.190890351446105e12; 1.012812302840952e12;;], [0.0], Int64[], 1, :u3D, LowLevelFEM.Problem("test-twist-3D", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 2648))

In [11]:
FEM.probe(q, 10, 1, 0)

3-element Vector{Float64}:
 0.0
 0.0
 6.858598415224617e11

In [12]:
FEM.showDoFResults(problem, q, :uvec, visible=true)
FEM.showDoFResults(problem, q1, :uvec, visible=true)

2

In [13]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [14]:
gmsh.finalize()